# Описание нотбука

В этом нотбуке тестируются несколько разных моделей на небольших данных.

# 0. Установка главной библиотеки

**Тут было очень больно** для дальнейшей работы нам нужна библа scikit-surprise
1. Т.к. библиотека использует cpython(как я понял из-за возникающих ошибок), а cpython это c++, то сначала необходимо установить Microsoft C++ Build Tools: https://visualstudio.microsoft.com/ru/visual-cpp-build-tools/

2. Затем устанавливаем numpy cython

!pip install numpy cython

3. Затем scikit-surprise

!pip install scikit-surprise

4. НЕОБХОДИМО ПЕРЕЗАГРУЗИТЬ ПК!!!

**Библиотеку установил, но дальше какая-то дичь, пока не решил, надо будет разбираться(**

In [41]:
import os

import time
import datetime
import random

import numpy as np
import pandas as pd
import scipy.sparse as sp

from surprise import Reader
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import Dataset, accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold
from surprise import NormalPredictor
from surprise import BaselineOnly
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNBaseline
from surprise import SlopeOne
from surprise import CoClustering



import warnings
warnings.filterwarnings('ignore')

# 1. Загрузка усеченных данных (для теста функционала модели)

In [74]:
%%time
interactions = pd.read_csv(
    "[test_models]_dataset_ratings.dat",
    sep="::",
    engine="python",
    header=None,
    names=['userId','movieId','rating','timestamp'],
)
print(interactions.shape)
interactions.head(3)

(1000209, 4)
CPU times: total: 3.62 s
Wall time: 3.61 s


,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968


In [3]:
%%time
movies_metadata = pd.read_csv(
    "[test_models]_dataset_movies.dat",
    sep="::",
    engine="python",
    header=None,
    names=['movieId','title','genres'],
    encoding='latin-1'
)
print(movies_metadata.shape)
movies_metadata.head(3)

(3883, 3)
CPU times: total: 15.6 ms
Wall time: 14.8 ms


,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [4]:
print(f'''interactions: {interactions.shape}
movies_metadata: {movies_metadata.shape}''')

interactions: (1000209, 4)
movies_metadata: (3883, 3)


# 2. ПРЕПРОЦЕССИНГ

In [9]:
reader = Reader(rating_scale=(1, 5))

In [21]:
data = Dataset.load_from_df(interactions[['userId', 'movieId', 'rating']], reader)

In [27]:
trainset, testset = train_test_split(data, test_size=.25)

# 3. Тестирование разным моделей для рексиса

# SVD

In [57]:
svd = SVD()

start = time.time()

cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
svd.fit(trainset)

predictions = svd.test(testset)

print("\nTest Set Error\n--------------")
accuracy.mae(predictions)
print("--------------\nFinished in {:.3f} sec.".format(time.time()-start))

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8722  0.8750  0.8759  0.8716  0.8718  0.8733  0.0018  
MAE (testset)     0.6838  0.6867  0.6879  0.6837  0.6846  0.6854  0.0017  
Fit time          6.62    6.74    6.70    6.72    6.75    6.71    0.05    
Test time         1.65    1.11    1.44    1.11    1.46    1.35    0.21    

Test Set Error
--------------
MAE:  0.6888
--------------
Finished in 53.031 sec.


# Неотрицательная матричная факторизация

**NMF** - Совместная фильтрация на основе факторизации матрицы

In [65]:
algo = NMF()

start = time.time()

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
algo.fit(trainset)

predictions = algo.test(testset)

print("\nTest Set Error\n--------------")
accuracy.mae(predictions)
print("--------------\nFinished in {:.3f} sec.".format(time.time()-start))

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9159  0.9146  0.9171  0.9199  0.9204  0.9176  0.0022  
MAE (testset)     0.7237  0.7224  0.7253  0.7261  0.7279  0.7251  0.0019  
Fit time          12.75   12.88   12.80   12.85   12.86   12.83   0.05    
Test time         1.34    0.96    0.96    0.95    0.97    1.04    0.15    

Test Set Error
--------------
MAE:  0.7246
--------------
Finished in 87.984 sec.


# Обычный KNN

**KNNBasic** - самая простая совместная фильтрация

In [67]:
algo = KNNBasic()

start = time.time()

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
algo.fit(trainset)

predictions = algo.test(testset)

print("\nTest Set Error\n--------------")
accuracy.mae(predictions)
print("--------------\nFinished in {:.3f} sec.".format(time.time()-start))

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9233  0.9215  0.9212  0.9249  0.9229  0.9228  0.0013  
MAE (testset)     0.7281  0.7262  0.7257  0.7296  0.7275  0.7274  0.0014  
Fit time          27.89   27.86   28.65   27.71   28.03   28.03   0.33    
Test time         77.55   75.78   76.68   73.17   74.37   75.51   1.57    
Computing the msd similarity matrix...
Done computing similarity matrix.

Test Set Error
--------------
MAE:  0.7286
--------------
Finished in 633.976 sec.


# KNNWithMeans

**KNNWithMeans** - реализует совместную фильтрацию, которая учитывает среднее значение рейтинга каждого пользователя

In [68]:
algo = KNNWithMeans()

start = time.time()

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
algo.fit(trainset)

predictions = algo.test(testset)

print("\nTest Set Error\n--------------")
accuracy.mae(predictions)
print("--------------\nFinished in {:.3f} sec.".format(time.time()-start))

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9295  0.9308  0.9272  0.9284  0.9312  0.9294  0.0015  
MAE (testset)     0.7381  0.7392  0.7378  0.7382  0.7401  0.7387  0.0009  
Fit time          28.26   28.45   28.62   28.36   28.33   28.41   0.12    
Test time         77.39   76.66   75.14   75.18   75.71   76.02   0.88    
Computing the msd similarity matrix...
Done computing similarity matrix.

Test Set Error
--------------
MAE:  0.7392
--------------
Finished in 640.283 sec.


# KNNBaseline

**KNNBaseline** - Совместная фильтрация с учетом базовых рейтингов

In [70]:
algo = KNNBaseline()

start = time.time()

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
algo.fit(trainset)

predictions = algo.test(testset)

print("\nTest Set Error\n--------------")
accuracy.mae(predictions)
print("--------------\nFinished in {:.3f} sec.".format(time.time()-start))

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8959  0.8964  0.8966  0.8933  0.8946  0.8954  0.0013  
MAE (testset)     0.7077  0.7073  0.7075  0.7050  0.7062  0.7067  0.0010  
Fit time          29.02   28.99   29.81   29.61   29.35   29.35   0.32    
Test time         76.87   76.88   78.00   77.69   78.09   77.51   0.53    
Estimating biases using als...
Computing th

# SlopeOne

**SlopeOne** - простой, но точный алгоритм совместной фильтрации

In [71]:
algo = SlopeOne()

start = time.time()

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
algo.fit(trainset)

predictions = algo.test(testset)

print("\nTest Set Error\n--------------")
accuracy.mae(predictions)
print("--------------\nFinished in {:.3f} sec.".format(time.time()-start))

Evaluating RMSE, MAE of algorithm SlopeOne on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9088  0.9079  0.9064  0.9053  0.9045  0.9066  0.0016  
MAE (testset)     0.7162  0.7148  0.7149  0.7135  0.7127  0.7144  0.0012  
Fit time          10.63   10.76   10.52   10.68   10.59   10.64   0.08    
Test time         29.33   28.97   29.07   28.71   29.19   29.05   0.21    

Test Set Error
--------------
MAE:  0.7144
--------------
Finished in 247.291 sec.


# CoClustering

**CoClustering** - Алгоритм совместной фильтрации на основе совместной кластеризации

In [72]:
algo = CoClustering()

start = time.time()

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
algo.fit(trainset)

predictions = algo.test(testset)

print("\nTest Set Error\n--------------")
accuracy.mae(predictions)
print("--------------\nFinished in {:.3f} sec.".format(time.time()-start))

Evaluating RMSE, MAE of algorithm CoClustering on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9139  0.9165  0.9153  0.9143  0.9139  0.9148  0.0010  
MAE (testset)     0.7165  0.7183  0.7178  0.7156  0.7165  0.7169  0.0010  
Fit time          16.84   16.80   16.91   16.87   16.86   16.86   0.04    
Test time         0.58    1.29    0.56    1.29    0.57    0.86    0.35    

Test Set Error
--------------
MAE:  0.7167
--------------
Finished in 110.656 sec.
